In [2]:
using JLD, ExponentialQueues
include("../RD.jl")
include("../RDnew.jl")
include("../lattice.jl")


#include("run_tests.jl")


gen_hex_lattice (generic function with 1 method)

In [3]:
#stats = Measurer(M; times=0:0.01:T)
#run_RD!(s, M, T; stats = stats)

function build_model_state(L)
    nn,centers = gen_hex_lattice(L)
    N = size(nn,1)
    memb0 = zeros(Int,N,4)	
    unit_length = 1.0

    theta = 0.2126944621086619#Stot/V


    Ac = unit_length^2#area associated to each cell, set unit lengthscale
    Stot = N*Ac #area of each cell is assumed to be 1, setting unit lengthscale
    V = Stot/theta
    d_timescale = 1.0#this sets unit timescale
    dA = d_timescale
    dB = d_timescale
    dEA = d_timescale
    dEB = d_timescale
    #rates in the theory (do not correspond excatly to those used in the simulations), some slight dimensional changes are needed
    kAa_th = 1.0
    kAd_th = 1.0*kAa_th
    kAc_th = 1.0
    kBa_th = 1.0
    kBd_th = 1.0*kBa_th
    kBc_th = logrange(logrange(0.1,10,17)[4],logrange(0.1,10.0,17)[14],23)[13]*kAc_th
    KMM_th = 1.0
    #rates to implement
    kAc = kAc_th
    kBc = kBc_th
    kAa = kAa_th/V
    kAd = kAd_th
    kBa = kBa_th/V
    kBd = kBd_th
    KMM = KMM_th*Ac

    M = Model(nn, centers, dA, dB, dEA, dEB, kAc, kBc, kAa, kAd, kBa, kBd, KMM, 0.0)

    N = size(M.neig, 1)
    totmol = N * 10
    totA = 0.5 * totmol
    totB = 0.5 * totmol
    theta = 0.2126944621086619#Stot/V

    nA, nB, nEA, nEB = fill(0,N), fill(0,N), fill(0,N), fill(0,N)

    for i in 1:totA
        nA[rand(1:N)] += 1
    end
    for i in 1:totB
        nB[rand(1:N)] += 1	
    end
    EA_tot_n = floor(Int, 0.1*N)
    EB_tot_n = floor(Int, 0.1*N)
    EA_mem = floor(Int, EA_tot_n*(theta/(kAd_th/kAa_th))*(totA/Stot)/(1+((theta/(kAd_th/kAa_th))*(totA/Stot))))
    EB_mem = floor(Int, EB_tot_n*(theta/(kBd_th/kBa_th))*(totB/Stot)/(1+((theta/(kBd_th/kBa_th))*(totB/Stot))))
    for i in 1:EA_mem
        nEA[rand(1:N)] += 1
    end
    for i in 1:EB_mem
        nEB[rand(1:N)] += 1
    end
    cytoEA = EA_tot_n - EA_mem
    cytoEB = EB_tot_n - EB_mem

    s = State(nA, nB, nEA, nEB, Ref(cytoEA), Ref(cytoEB))
    M,s
end

build_model_state (generic function with 1 method)

In [4]:
using Profile

T = 20000.0
L = 50

M,s = build_model_state(L)

using ProgressMeter

function ProgressMeasurer(T)
    p = Progress(100)
    function stat(t, s)
        update!(p, floor(Int, 100*t/T))
    end
end

stats = ProgressMeasurer(T)

run_RD!(s, M, T; stats)

starting simulation, 10407 events in the queue


Progress:   6%|██▌                                      |  ETA: 0:06:17

InterruptException: InterruptException:

In [24]:
include("run_tests.jl")

([0 12 0 0; 0 7 0 0; … ; 0 5 0 0; 0 13 0 0], [250, 72])